In [12]:
import json

file = "time_related_queries.json"

with open(f"dataset/{file}") as f:
    samples = json.load(f)
    
with open(f"dataset_parsed/{file}") as f:
    parsed_samples = json.load(f)
    
with open(f"generated_ppls_parsed/{file}") as f:
    parsed_ppls = json.load(f)
    
with open(f"generated_ppls/{file}") as f:
    baseline_ppls = json.load(f)
    
with open(f"results/gold/{file}") as f:
    gold = json.load(f)
    
with open(f"results/baseline/{file}") as f:
    baseline = json.load(f)
    
with open(f"results/baseline-parsed/{file}") as f:
    parsed = json.load(f)
    
for i in range(len(samples)):
    if samples[i]["time_range"][0]!= parsed_samples[i]["start"] or samples[i]["time_range"][1]!= parsed_samples[i]["end"]:
        print(i, samples[i]["question"], samples[i]["time_range"], (parsed_samples[i]["start"], parsed_samples[i]["end"]))

9 Show me the weekly trend of the total number of orders. [None, None] ('2025-05-20 03:53:30', '2025-05-27 03:53:30')
81 Generate a weekly trend of successful login events in the sso_log index. [None, None] ('2025-05-20 03:53:30', '2025-05-27 03:53:30')


In [3]:
from utils import eval_execution_accuracy, simple_parse

corrects = []
for i in range(len(samples)):
    gt_sample = gold[i]
    eval_sample = baseline[i]
        # Check for syntax errors
    if eval_sample["data_rows"] == "ERROR":
        continue

    # Get data rows
    gt_rows = gt_sample["data_rows"]
    eval_rows = eval_sample["data_rows"]

    try:
        if eval_execution_accuracy(
            simple_parse(eval_rows, eval_sample["schema"]),
            simple_parse(gt_rows, gt_sample["schema"]),
        ):
            corrects.append(i)
    except:
        print(i)
        pass
    
len(corrects)

5
47
63
72


46

In [4]:
from compare_results import compare_results

compare_results(f"results/gold/{file}", f"results/baseline/{file}", "tmp.json")

In [5]:
corrects_parsed = []
for i in range(len(samples)):
    gt_sample = gold[i]
    eval_sample = parsed[i]
        # Check for syntax errors
    if eval_sample["data_rows"] == "ERROR":
        continue

    # Get data rows
    gt_rows = gt_sample["data_rows"]
    eval_rows = eval_sample["data_rows"]

    try:
        if eval_execution_accuracy(
            simple_parse(eval_rows, eval_sample["schema"]),
            simple_parse(gt_rows, gt_sample["schema"]),
        ):
            corrects_parsed.append(i)
    except:
        pass

In [6]:
set(corrects) - set(corrects_parsed), set(corrects_parsed) - set(corrects)

({0, 3, 6, 10, 11, 21, 28, 34, 39, 53, 65, 68, 73}, {17, 47, 63, 72})

In [9]:
for idx in {68}:
    print(samples[idx]["question"])
    print(samples[idx]["now"])
    print(parsed_samples[idx]["question"])
    print(parsed_ppls[idx]["query"])
    print(baseline_ppls[idx]["query"])
    print(samples[idx]["query"])
    print("*"*100)

Find the ratio of logs where the request URL contains the word "recommendations" in the last week.
TIMESTAMP('2025-05-27 03:53:30')
Find the ratio of logs where the request URL contains the word "recommendations"
source=sso_log| where SUBSTRING(`http.url`, 1) = 'recommendations' | stats COUNT() as count_with_recommendations, COUNT() as total_count | eval ratio = count_with_recommendations / total_count
source=sso_log| where @timestamp > DATE_SUB(NOW(), INTERVAL 1 WEEK) | eval contains_recommendations = IF(LIKE(`http.url`, '%recommendations%'), 1, 0) | stats AVG(contains_recommendations) as ratio_recommendations
source=sso_log 
| where @timestamp > DATE_SUB(NOW(), INTERVAL 1 WEEK)
| eval is_recommendations = IF(LIKE(http.url, '%recommendations%'), 1.0, 0.0)
| stats AVG(is_recommendations) as recommendations_ratio
****************************************************************************************************
